# Solutions to Workshop Exercises

## **Exercise 3.1 Solution**

**Query**: Find the top 10 subreddits discussing "election" the most between 2013 and 2015\.

```
SELECT subreddit, COUNT(*) as comment_count
FROM `pushshift.rt_reddit.comments`
WHERE body LIKE '%election%'
GROUP BY subreddit
ORDER BY comment_count DESC
LIMIT 10;
```

**Explanation**:

- `body LIKE '%election%'`: Filters comments containing the word "election".

## **Exercise 3.2 Solution**

**Query**: Identify the most active users (authors) discussing "election".

```
SELECT author, COUNT(*) as comment_count
FROM `pushshift.rt_reddit.comments`
WHERE body LIKE '%election%'
GROUP BY author
ORDER BY comment_count DESC
LIMIT 10;

```

## **Exercise 3.3 Solution**

**Note**: Sentiment analysis requires additional tools or libraries.

**Approach**:

- Export comments containing "election" to a CSV file.  
- Use Python with a library like `TextBlob` or `NLTK` to perform sentiment analysis.

```py
import csv
from textblob import TextBlob

# Read comments from CSV
with open('election_comments.csv', 'r', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    sentiments = []
    for row in reader:
        analysis = TextBlob(row['body'])
        sentiments.append({
            'comment_id': row['id'],
            'sentiment': analysis.sentiment.polarity
        })

# Analyze average sentiment
average_sentiment = sum([s['sentiment'] for s in sentiments]) / len(sentiments)
print(f"Average sentiment: {average_sentiment}")
```

## **Exercise 4 Solution**

**Objective**: Compare Reddit discussions with Wikipedia pageviews.

Return to [Challenge Exercise 4](?tab=t.0#heading=h.p8i4kgv2vhyk)

1. Identify Google Trends terms from the past week  
2. Query the Reddit dataset from 2015-2016  
3. Join with Reddit dataset with Google Trends to find articles that contain the terms from the trends

**Query Steps**:

1. **Get Top Trending terms in the past week:**

```
Top trending terms in the past week:

-- Extract the top trending terms from the past week
SELECT
  refresh_date AS date,
  term,
  SUM(score) AS trend_score
FROM
  `bigquery-public-data.google_trends.top_terms`
WHERE
  refresh_date BETWEEN DATE_SUB(CURRENT_DATE(), INTERVAL 7 DAY) AND CURRENT_DATE()
GROUP BY
  date, term
ORDER BY
  trend_score DESC
LIMIT 50;  -- Adjust the limit to see more or fewer top trends

```

Full Solution:

```
-- Step 1: Extract Google Trends data for the week from Nov 1 to Nov 7, 2024
WITH google_trends AS (
  SELECT
    refresh_date AS date,
    term,
    SUM(score) AS trend_score
  FROM
    `bigquery-public-data.google_trends.top_terms`
  WHERE
    refresh_date BETWEEN '2024-11-01' AND '2024-11-07'
  GROUP BY
    date, term
),

-- Step 2: Extract Reddit comments from 2016 containing election-related keywords
reddit_data AS (
  SELECT
    DATE(created_utc) AS date,
    body,
    subreddit,
    COUNT(*) OVER (PARTITION BY DATE(created_utc)) AS reddit_comment_count
  FROM
    `pushshift.rt_reddit.comments`
  WHERE
    subreddit IN ('politics', 'The_Donald', 'news', 'worldnews')  -- Example subreddits
)

-- Step 3: Join Google Trends data with Reddit comments data from 2016 to find comments containing the trending terms
SELECT
  reddit_data.date AS reddit_date,
  reddit_data.subreddit,
  reddit_data.body AS comment,
  google_trends.term AS trending_term,
  google_trends.trend_score
FROM
  reddit_data
INNER JOIN
  google_trends
ON
  LOWER(reddit_data.body) LIKE CONCAT('%', LOWER(google_trends.term), '%')  -- Case-insensitive match of trending terms in comments
ORDER BY
  reddit_data.date DESC;

```

Additional Solutions

2. **Get Daily Comment Counts from Reddit**:

```
SELECT
 DATE(TIMESTAMP(created_utc)) as date,
 COUNT(*) as reddit_comment_count
FROM `pushshift.rt_reddit.comments`
WHERE body LIKE '%election%'
GROUP BY date
```

3. **Get Daily Pageviews from Wikipedia**:

```
SELECT
 DATE(datehour) as date,
 SUM(views) as wiki_pageviews
FROM `bigquery-public-data.wikipedia.pageviews_2016`
WHERE REGEXP_CONTAINS(title, r'(?i)election')
AND DATE(datehour)<>current_date()
GROUP BY date

```

4. **Join the Two Results**:

```
WITH reddit_data AS (
  -- Reddit query here
),
wiki_data AS (
  -- Wikipedia query here
)
SELECT
  reddit_data.date,
  reddit_data.reddit_comment_count,
  wiki_data.wiki_pageviews
FROM reddit_data
JOIN wiki_data ON reddit_data.date = wiki_data.date
ORDER BY reddit_data.date;
```

Complete solution:

```
WITH reddit_data AS (
SELECT
 DATE(TIMESTAMP(created_utc)) as date,
 COUNT(*) as reddit_comment_count
FROM `pushshift.rt_reddit.comments`
WHERE body LIKE '%election%'
GROUP BY date
),
wiki_data AS (
SELECT
 DATE(datehour) as date,
 SUM(views) as wiki_pageviews
FROM `bigquery-public-data.wikipedia.pageviews_2015`
WHERE REGEXP_CONTAINS(title, r'(?i)election')
AND DATE(datehour)<>current_date()
GROUP BY date
)
SELECT
  reddit_data.date,
  reddit_data.reddit_comment_count,
  wiki_data.wiki_pageviews
FROM reddit_data
JOIN wiki_data ON reddit_data.date = wiki_data.date
ORDER BY reddit_data.date;
```

## **Exercise 5.1 Solution**

**Query**: Find companies in California in the Tech industry.

```
SELECT *
FROM `your_project.dnb_data.companies`
WHERE state = 'CA'
  AND industry = 'Technology';
```

## **Exercise 6 Solution**

**Modified Python Script to Save Data**:

```py
import praw
import csv
import datetime

reddit = praw.Reddit(
    client_id='YOUR_CLIENT_ID',
    client_secret='YOUR_CLIENT_SECRET',
    user_agent='WorkshopScript by /u/YourRedditUsername'
)

subreddit = reddit.subreddit('all')
with open('recent_election_comments.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['comment_id', 'author', 'body', 'created_utc']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    for comment in subreddit.stream.comments(skip_existing=True):
        if 'election' in comment.body.lower():
            writer.writerow({
                'comment_id': comment.id,
                'author': str(comment.author),
                'body': comment.body,
                'created_utc': datetime.datetime.utcfromtimestamp(comment.created_utc).isoformat()
            })
```

## **Exercise 7 Solution**

**Load Scraped Data into BigQuery**:

1. **Prepare the Data**:  
     
   - Ensure `recent_election_comments.csv` is properly formatted.

   

2. **Load Data**:  
     
   - In BigQuery, create a new table under your dataset.  
   - Use **Create Table From**: Upload.  
   - Select your CSV file.  
   - Let BigQuery auto-detect schema.

   

3. **Verify Data**:  
     
   - Run a simple `SELECT` query to ensure data integrity.

## **Exercise 8.1 Solution**

**Query**: Find mentions of companies in Reddit comments.

```
SELECT
  c.company_name,
  COUNT(r.comment_id) as mention_count
FROM `your_project.dnb_data.companies` c
JOIN `your_project.your_dataset.reddit_comments` r
ON REGEXP_CONTAINS(r.body, CONCAT('(?i)', c.company_name))
GROUP BY c.company_name
ORDER BY mention_count DESC;
```

**Note**:

- Ensure that both datasets are properly loaded and accessible.  
- This query can be resource-intensive due to regular expression matching on large text fields.
